<a href="https://colab.research.google.com/github/chengulatj/decoding_VINs/blob/main/VIN_Decoding_AV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from google.colab import drive
import pandas as pd
import requests

# Step 2: Read the API link securely from a file
with open('/content/drive/MyDrive/VINLINK.txt') as f:
    for line in f:
        if '=' in line:
            parts = line.strip().split('=')  # Split the line by '='
            # Safely access the key and value, handling potential extra '='
            key = parts[0] if parts else None
            value = parts[1] if len(parts) > 1 else None
            if key == 'VINLINK':
                VINLINK = value
                break  # Exit loop after finding VINLINK

# Step 3: Define a function to decode a VIN using the NHTSA API
def decode_vin(vin):
    # Use the secure API link read from the file
    url = f"{VINLINK}{vin}?format=json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Step 4: Load the dataset containing VINs
df = pd.read_csv("/content/drive/MyDrive/vin_nos.csv")

# Step 5: Create an empty list to store the decoded VIN information
decoded_vins = []

# Step 6: Iterate over the VIN numbers in the dataset and decode them
for vin in df['VIN']:
    decoded_info = decode_vin(vin)
    if decoded_info and 'Results' in decoded_info:
        # Append the decoded information to the list
        decoded_vins.append(decoded_info['Results'][0])

# Step 7: Convert the list of decoded information to a DataFrame
decoded_vins_df = pd.DataFrame(decoded_vins)

# Step 8: Add a temporary unique identifier to each DataFrame
df['temp_id'] = range(1, len(df) + 1)
decoded_vins_df['temp_id'] = range(1, len(decoded_vins_df) + 1)

# Step 9: Merge the DataFrames using VIN and the temporary identifier
merged_df = pd.merge(df, decoded_vins_df, on=['VIN', 'temp_id'], how='inner')

# Step 10: Drop the temporary identifier
merged_df.drop('temp_id', axis=1, inplace=True)

# Step 11: Save the merged DataFrame to a new CSV file
merged_df.to_csv("/content/drive/MyDrive/decoded_vins.csv", index=False)

print("Decoded VINs saved to 'decoded_vins.csv'")


Decoded VINs saved to 'decoded_vins.csv'
